<a href="https://colab.research.google.com/github/Kilvia/inception-network/blob/main/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Inception Module

- `strides`는 모두 1로 설정해야 output size가 맞게 나올 것이다.



<img src="https://user-images.githubusercontent.com/37704174/108797724-369ef800-75cf-11eb-987a-926657daf61e.JPG" width="400" height="400"/>  



### Inception Network Architecture
- 아래의 표에 있는대로 구성을 하면 이러한 구조의 모델이 만들어진다.

<img src="https://user-images.githubusercontent.com/37704174/108797713-2dae2680-75cf-11eb-8593-5a1bf50549da.png" width="400" height="400"/>  

### Specification


<img src="https://user-images.githubusercontent.com/37704174/109782740-8d9a7200-7c4c-11eb-9999-d859b67dc468.JPG" width="600" height="600"/>  

- 이 표에 있는 대로 위에서부터 layer 및 Inception module을 쌓아야 한다.
- 모든 `Conv2D`와 `MaxPool2D`의 padding 옵션은 `same`으로 설정해야 output size가 올바르게 나올 것이다.
- 표 3행을 보면 convolution의 depth가 2이다. 이것은 `Conv2D` layer가 2개 있다는 뜻이고, 각각의 filter의 수는 64, 192로 설정해야 한다.
- Inception module의 hyperparameter 설명
  - #1x1: 맨 왼쪽 1x1 convolution layer의 filter 수
  - #3x3 reduce: 3x3 convolution layer 이전에 나오는 1x1 convolution layer의 filter 수
  - #3x3: 3x3 convolution layer의 filter 수
  - #5x5 reduce: 5x5 convolution layer 이전에 나오는 1x1 convolution layer의 filter 수
  - #5x5: 5x5 convolution layer의 filter 수
  - pool proj: max pooling 다음에 나오는 1x1 convolution layer의 filter 수

## Import Modules

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Concatenate, Flatten, Add, MaxPool2D, GlobalAveragePooling2D, Conv1D
### 자유롭게 import

## GPU Setting

In [ ]:
"""
Make sure your runtime type is GPU!
"""
physical_devices = tf.config.list_physical_devices('GPU')
print('Num_GPUs:{}, List:{}'.format(len(physical_devices), physical_devices))

Num_GPUs:0, List:[]


## 1. Implement Inception Network


### Inception Module

In [ ]:
def InceptionModule(inputs, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pool):
    """
    inputs: input of Inception module
    f_ : number of filters in the table above
    
    """
    x_1_1 = MaxPool2D(pool_size=(3,3), strides=(1,1), padding='same')(inputs)
    x_2_1 = Conv2D(filters=f_5x5_r, kernel_size=(1,1), padding='same', activation='relu')(inputs)
    x_3_1 = Conv2D(filters=f_3x3_r, kernel_size=(1,1), padding='same', activation='relu')(inputs)
    
    x_1_2 = Conv2D(filters=f_pool, kernel_size=(1,1), padding='same', activation='relu')(x_1_1)
    x_2_2 = Conv2D(filters=f_5x5, kernel_size=(5,5), padding='same', activation='relu')(x_2_1)
    x_3_2 = Conv2D(filters=f_3x3, kernel_size=(3,3), padding='same', activation='relu')(x_3_1)
    x_4 = Conv2D(filters=f_1x1, kernel_size=(1,1), padding='same', activation='relu')(inputs)
    
    outputs = Concatenate(axis=-1)([x_1_2, x_2_2, x_3_2, x_4])
    
    #######################################
    return outputs

### Inception Network

In [ ]:
def InceptionNet():
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))
    
    x = Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding='same')(inputs)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
    x = Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

    x = InceptionModule(x, 64, 96, 128, 16, 32, 32)
    x = InceptionModule(x, 128, 128, 192, 32, 96, 64)

    x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

    x = InceptionModule(x, 192, 96, 208, 16, 48, 64)
    x = InceptionModule(x, 160, 112, 224, 24, 64, 64)
    x = InceptionModule(x, 128, 128, 256, 24, 64, 64)
    x = InceptionModule(x, 112, 144, 288, 32, 64, 64)
    x = InceptionModule(x, 256, 160, 320, 32, 128, 128)
    
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

    x = InceptionModule(x, 256, 160, 320, 32, 128, 128)
    x = InceptionModule(x, 384, 192, 384, 48, 128, 128)

    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='linear')(x)
    
    outputs = Dense(10, activation='softmax')(x)
    
    #########################################################################
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
my_inception = InceptionNet()

### Model Summary

아래 셀을 실행 했을 때, 표의 각각에 해당하는 output size가 맞게 나와야 한다.

In [ ]:
my_inception.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   9472        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 16, 64)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 16, 192)  110784      ['max_pooling2d[0][0]']          
                                                                                              

## 2. Dataset

### Training Data

아래의 데이터를 이용한다.

In [ ]:
### DO NOT MODIFY ###
cifar10 = tf.keras.datasets.cifar10
#Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Split train set into train/valid set
from sklearn import model_selection
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train, y_train,test_size=0.1)

### Data Preprocessing

원하는 대로 데이터를 전처리 해주면 된다.

In [ ]:
x_train = tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255.0)(x_train)
x_test = tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255.0)(x_test)
x_valid = tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255.0)(x_valid)

# x_train = tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical')(x_train)
# x_train = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(x_train)


###############################################

## 3. Training

### Model Compile

Optimizer, Loss function을 알맞게 설정한다. <br>
Callbacks도 자유롭게 사용한다.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optim_fn = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

my_inception.compile( optimizer=optim_fn,
                     loss=loss_fn,
                     metrics=['accuracy'] )

In [ ]:
import math
import os

checkpoint_dir = './training_checkpoints_cnn'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

def decay(epoch, steps=100):
  # Change Learning rate
    learn_rate = 0.01
    drop = 0.96
    epoch_drop = 8

    curr_learn_rate = learn_rate * math.pow(drop, math.floor((1+epoch)/epoch_drop))
    return curr_learn_rate

callbacks = [
    # tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only = True),
    tf.keras.callbacks.LearningRateScheduler(decay, verbose=1)
]

### Model Training

hyperparameter를 적절히 설정한다. (epochs 등..)

In [ ]:
## Q5. Set hyperparameters & training ##
hist = my_inception.fit(x=x_train, 
                        y=y_train, 
                        validation_data=(x_valid, y_valid),
                        callbacks=callbacks,
                        epochs=25
                        )
########################################

Epoch 1/15
1407/1407 [==============================] - 1248s 887ms/step - loss: 2.2494 - accuracy: 0.1362 - val_loss: 2.1439 - val_accuracy: 0.1756 - lr: 1.0000e-04
Epoch 2/15
1407/1407 [==============================] - 1242s 883ms/step - loss: 2.0273 - accuracy: 0.2268 - val_loss: 1.9574 - val_accuracy: 0.2272 - lr: 1.0000e-04
Epoch 3/15
1407/1407 [==============================] - 1253s 891ms/step - loss: 1.8784 - accuracy: 0.2961 - val_loss: 1.7820 - val_accuracy: 0.3332 - lr: 1.0000e-04
Epoch 4/15
1407/1407 [==============================] - 1256s 893ms/step - loss: 1.7493 - accuracy: 0.3526 - val_loss: 1.7201 - val_accuracy: 0.3644 - lr: 1.0000e-05
Epoch 5/15
1407/1407 [==============================] - 1262s 897ms/step - loss: 1.7212 - accuracy: 0.3638 - val_loss: 1.7210 - val_accuracy: 0.3640 - lr: 1.0000e-05
Epoch 6/15
1407/1407 [==============================] - 1246s 886ms/step - loss: 1.7021 - accuracy: 0.3713 - val_loss: 1.7064 - val_accuracy: 0.3670 - lr: 1.0000e-05
Epoc

## 참고 >

조교가 학습한 모델의 validation accuracy를 그래프로 나타내 보았다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist.history['val_accuracy'])
plt.title('Validation Accuracy', fontsize=15)
plt.xlabel('epochs', fontsize=15)
plt.ylabel('Acc.', fontsize=15)

plt.show()

## Test Accuracy


In [ ]:
my_inception.evaluate(x_test, y_test)